In [1]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
import re 
import string
import nltk
from nltk.stem import WordNetLemmatizer
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_columns', 500)

import numpy as np
from sklearn.model_selection import train_test_split
from nltk.chunk import ne_chunk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag
from nltk.tokenize import MWETokenizer

## Exploratory Data Analysis

Read in the news data, which consists of two parts: first part is the Daily World News, and only the top 25 headlines are considered for a single date. Second part is the Dow Jones Industrial Average (DJIA) stock data (target variable) measured by 0 and 1: "1" when DJIA Adj Close value rose or stayed as the same, "0" when DJIA Adj Close value decreased.

In [2]:
data = pd.read_csv('Combined_News_DJIA.csv')

In [3]:
data.head(1)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25
0,2008-08-08,0,"""Georgia 'downs two Russian warplanes' as countries move to brink of war""",'BREAKING: Musharraf to be impeached.','Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)',"'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'","""Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing""",'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.',"""Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side""","""The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it.""","'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]'",'Did the U.S. Prep Georgia for War with Russia?','Rice Gives Green Light for Israel to Attack Iran: Says U.S. has no veto over Israeli military ops','Announcing:Class Action Lawsuit on Behalf of American Public Against the FBI',"""So---Russia and Georgia are at war and the NYT's top story is opening ceremonies of the Olympics? What a fucking disgrace and yet further proof of the decline of journalism.""","""China tells Bush to stay out of other countries' affairs""",'Did World War III start today?',"'Georgia Invades South Ossetia - if Russia gets involved, will NATO absorb Georgia and unleash a full scale war?'",'Al-Qaeda Faces Islamist Backlash',"'Condoleezza Rice: ""The US would not act to prevent an Israeli strike on Iran."" Israeli Defense Minister Ehud Barak: ""Israel is prepared for uncompromising victory in the case of military hostilities.""'",'This is a busy day: The European Union has approved new sanctions against Iran in protest at its nuclear programme.',"""Georgia will withdraw 1,000 soldiers from Iraq to help fight off Russian forces in Georgia's breakaway region of South Ossetia""",'Why the Pentagon Thinks Attacking Iran is a Bad Idea - US News &amp; World Report','Caucasus in crisis: Georgia invades South Ossetia',"'Indian shoe manufactory - And again in a series of ""you do not like your work?""'",'Visitors Suffering from Mental Illnesses Banned from Olympics',"""No Help for Mexico's Kidnapping Surge"""


Prepare the data by adding lag up to three days. Adding lag features will help to evaluate the impact of news under broader time window. 

In [4]:
# increment date by 1/2/3
data['newdate1'] = pd.to_datetime(data['Date']).apply(pd.DateOffset(1)).dt.date
data['newdate2'] = pd.to_datetime(data['Date']).apply(pd.DateOffset(2)).dt.date
data['newdate3'] = pd.to_datetime(data['Date']).apply(pd.DateOffset(3)).dt.date

In [5]:
data['Date'] = pd.to_datetime(data['Date']).dt.date

In [6]:
data['lag1'] = np.nan
data['lag2'] = np.nan
data['lag3'] = np.nan

for i in range(len(data)):
    for j in range(len(data)):
        if data.loc[i, 'newdate1'] == data.loc[j, 'Date']:
            data.loc[i, 'lag1'] = int(data.loc[j, 'Label'])
        if data.loc[i, 'newdate2'] == data.loc[j, 'Date']:
            data.loc[i, 'lag2'] = int(data.loc[j, 'Label'])
        if data.loc[i, 'newdate3'] == data.loc[j, 'Date']:
            data.loc[i, 'lag3'] = int(data.loc[j, 'Label'])

In [7]:
data.head(1)

,Date,Label,Top1,Top2,Top3,Top4,Top5,Top6,Top7,Top8,Top9,Top10,Top11,Top12,Top13,Top14,Top15,Top16,Top17,Top18,Top19,Top20,Top21,Top22,Top23,Top24,Top25,newdate1,newdate2,newdate3,lag1,lag2,lag3
0,2008-08-08,0,"""Georgia 'downs two Russian warplanes' as countries move to brink of war""",'BREAKING: Musharraf to be impeached.','Russia Today: Columns of troops roll into South Ossetia; footage from fighting (YouTube)',"'Russian tanks are moving towards the capital of South Ossetia, which has reportedly been completely destroyed by Georgian artillery fire'","""Afghan children raped with 'impunity,' U.N. official says - this is sick, a three year old was raped and they do nothing""",'150 Russian tanks have entered South Ossetia whilst Georgia shoots down two Russian jets.',"""Breaking: Georgia invades South Ossetia, Russia warned it would intervene on SO's side""","""The 'enemy combatent' trials are nothing but a sham: Salim Haman has been sentenced to 5 1/2 years, but will be kept longer anyway just because they feel like it.""","'Georgian troops retreat from S. Osettain capital, presumably leaving several hundred people killed. [VIDEO]'",'Did the U.S. Prep Georgia for War with Russia?','Rice Gives Green Light for Israel to Attack Iran: Says U.S. has no veto over Israeli military ops','Announcing:Class Action Lawsuit on Behalf of American Public Against the FBI',"""So---Russia and Georgia are at war and the NYT's top story is opening ceremonies of the Olympics? What a fucking disgrace and yet further proof of the decline of journalism.""","""China tells Bush to stay out of other countries' affairs""",'Did World War III start today?',"'Georgia Invades South Ossetia - if Russia gets involved, will NATO absorb Georgia and unleash a full scale war?'",'Al-Qaeda Faces Islamist Backlash',"'Condoleezza Rice: ""The US would not act to prevent an Israeli strike on Iran."" Israeli Defense Minister Ehud Barak: ""Israel is prepared for uncompromising victory in the case of military hostilities.""'",'This is a busy day: The European Union has approved new sanctions against Iran in protest at its nuclear programme.',"""Georgia will withdraw 1,000 soldiers from Iraq to help fight off Russian forces in Georgia's breakaway region of South Ossetia""",'Why the Pentagon Thinks Attacking Iran is a Bad Idea - US News &amp; World Report','Caucasus in crisis: Georgia invades South Ossetia',"'Indian shoe manufactory - And again in a series of ""you do not like your work?""'",'Visitors Suffering from Mental Illnesses Banned from Olympics',"""No Help for Mexico's Kidnapping Surge""",2008-08-09,2008-08-10,2008-08-11,NaN,NaN,1.0


### NLP

Brief summary of natural language processing:

- **Word Similarity**: can be used for spell check, speech recognition
    
    Measured by Levenshtein distance: compute minimum steps taken from one word to another
   
    Tools: NLTK, TextBlob (tokenization, spell check, parts of speech tagging, sentiment...)
    
    
- **Document Similarity**: used to find similar documents out of a large number of documents; group similar documents, etc. 
    
    *1) Document-term matrix - Count Vectorizer: helps create a document-term matrix*
       - Downsides: Counts can be too simplistic, high counts can dominate, each word is treated equally however some terms might be more important than others
    
    *2) Document-term matrix - TF-IDF (Term Frequency ** Inverse Document Frequency - a metric that accounts for the issues of count vectorizer): gives more weights to rare words
    
    *3) Assumption - Bag of Words: each document is recognized as a bag of its words, where grammar and word order are disregarded, but multiplicity is kept*
    
    *4) Similarity Measurement - Cosine similarity: is a way to quantify similarity between documents (put each document in vector format, find the cosine of the angle between the documents)*
          
        

Text data is messy, so before NLP, we need to process the text to make it clean and easy to analyze. 

Preprocessing techniques:

- **Tokenization:** turn text into a meaningful format/units for analysis
    These units can be:
    1) words, 2) sentences, 3) n-grams, 4) other characters defined by regular expressions
- **Clean the data:** 
  *remove:* capital letters, punctuation, numbers, stop words
  *stemming/lemmatization:* 
  *chunking:* named entity recognition
  

In [8]:
# text processing
def combine_text (row):
    wordnet_lemmatizer = WordNetLemmatizer()
    combined = ""
    for i in range(0, len(row)):       
        text = ""
        text = str(row[i]).lower()
#         remove numbers
        text = re.sub('\w*\d\w*', ' ', text)
#         remove punctuations
        text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
#         compound term extraction
        mwe_tokenizer = MWETokenizer([('united','states')])
        text = mwe_tokenizer.tokenize(word_tokenize(text))
        text = ' '.join(text)
        text = CountVectorizer().build_tokenizer()(text)
        transformed_words = []  
# some text could not be processed by lemmatization, for example: russian -> russia, so manually transfer those words below:
        for x in text:     
            if x == "russian":
                x = "russia"
            if x == "chinese":
                x = 'china'
            if x == "indian":
                x = "india"
            if x == "korean":
                x = "korea"
            if x == "japanese":
                x = "japan"
            if x == "us" or x == 'usa' or x == 'america':
                x = "united_states"
            if x == "european":
                x = "europe"
            if x == 'scottish':
                x = 'scotland'
            if x == 'nigerian':
                x = 'nigeria'
            # word lemmatize gives more interpretable results
            transformed_words.append(wordnet_lemmatizer.lemmatize(x))
        res = ' '.join(transformed_words)        
        combined += res + " "
    return combined

df = pd.DataFrame()
df['text'] = data.iloc[:,2:27].apply(lambda row: combine_text(row), axis=1)
df[['Date','Label', 'newdate1', 'newdate2', 'newdate3','lag1','lag2','lag3']] = data[['Date','Label', 'newdate1', 'newdate2', 'newdate3','lag1','lag2','lag3']]
df.head(1)

,text,Date,Label,newdate1,newdate2,newdate3,lag1,lag2,lag3
0,georgia down two russia warplane a country move to brink of war breaking musharraf to be impeached russia today column of troop roll into south ossetia footage from fighting youtube russia tank are moving towards the capital of south ossetia which ha reportedly been completely destroyed by georgian artillery fire afghan child raped with impunity un official say this is sick three year old wa raped and they do nothing russia tank have entered south ossetia whilst georgia shoot down two russia jet breaking georgia invades south ossetia russia warned it would intervene on so side the enemy combatent trial are nothing but sham salim haman ha been sentenced to year but will be kept longer anyway just because they feel like it georgian troop retreat from osettain capital presumably leaving several hundred people killed video did the united_states prep georgia for war with russia rice give green light for israel to attack iran say united_states ha no veto over israeli military ops announcingclass action lawsuit on behalf of american public against the fbi sorussia and georgia are at war and the nyts top story is opening ceremony of the olympics what fucking disgrace and yet further proof of the decline of journalism china tell bush to stay out of other country affair did world war iii start today georgia invades south ossetia if russia get involved will nato absorb georgia and unleash full scale war alqaeda face islamist backlash condoleezza rice the united_states would not act to prevent an israeli strike on iran israeli defense minister ehud barak israel is prepared for uncompromising victory in the case of military hostility this is busy day the europe union ha approved new sanction against iran in protest at it nuclear programme georgia will withdraw soldier from iraq to help fight off russia force in georgia breakaway region of south ossetia why the pentagon think attacking iran is bad idea united_states news amp world report caucasus in crisis georgia invades south ossetia india shoe manufactory and again in series of you do not like your work visitor suffering from mental illness banned from olympics no help for mexico kidnapping surge,2008-08-08,0,2008-08-09,2008-08-10,2008-08-11,NaN,NaN,1.0


In [135]:
import pickle

with open('data_lagged.pickle', 'wb') as to_write:
    pickle.dump(df, to_write)

In [ ]:
# import pickle
# with open('data.pickle','rb') as read_file:
#     df = pickle.load(read_file)

In [9]:
topic_modeling = []

for row in range(0,len(df)):
    topic_modeling.append(df.iloc[row,0])

Tried Gensim...it doesn't give me the results I'm looking for

In [ ]:
# import gensim
# import pyLDAvis
# from pyLDAvis import gensim as gensimvis
# import spacy

# import logging
# from tqdm import tqdm
# from pprint import pprint

# from sklearn import datasets

# combined = []
# for row in range(0,len(data.index)):
#     testheadlines = ' '.join(str(x) for x in data.iloc[row,2:27])
#     text = gensim.utils.simple_preprocess(testheadlines, deacc=True)
#     res = ' '.join(text)        
#     combined.append(res)
    
# combined

#### Split train and test

In [10]:
df['Date'] = df['Date'].astype(str)
train = df[df['Date'] < '2015-01-01']
test = df[df['Date'] > '2014-12-31']

## Base Model - Logistic Regression

In [11]:
cv = CountVectorizer(stop_words='english')
X = cv.fit_transform(train['text'])
df_train = pd.DataFrame(X.toarray(), columns=cv.get_feature_names())
X.shape

(1611, 29727)

In [12]:
basicmodel = LogisticRegression()
basicmodel = basicmodel.fit(df_train, train['Label'])

X_test = cv.transform(test['text'])
predictions = basicmodel.predict(X_test)

/Users/zhaoze/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


##### Accuracy is 46%, with total train data 378

In [13]:
pd.crosstab(test["Label"], predictions, rownames=["Actual"], colnames=["Predicted"])

Predicted,0,1
Actual,,
0,73,113
1,91,101


The cell below will get a list of the names from our CountVectorizer and a list of the coefficients from our model, then combine the two lists into a Pandas dataframe.
Once that's made, we can sort it and check out the top 10 positive and negative coefficients.

In [14]:
basicwords = cv.get_feature_names()
basiccoeffs = basicmodel.coef_.tolist()[0]
coeffdf = pd.DataFrame({'Word' : basicwords, 
                        'Coefficient' : basiccoeffs})
coeffdf = coeffdf.sort_values(['Coefficient', 'Word'], ascending=[0, 1])


In [15]:
new_popular = coeffdf.iloc[:10,:]
new_popular

,Word,Coefficient
17654,nigeria,0.493259
11164,green,0.479936
23340,scrap,0.471042
20459,propaganda,0.449174
21926,resolution,0.448498
4486,chemical,0.448352
19028,past,0.447586
29199,wolf,0.441628
17094,mumbai,0.438808
14822,leak,0.427980


In [16]:
new_negative = coeffdf.tail(10)
new_negative 

,Word,Coefficient
27768,university,-0.463453
26299,tell,-0.482441
26878,total,-0.486529
24601,society,-0.490082
2613,begin,-0.509085
23048,sanction,-0.521942
21191,real,-0.530606
22824,run,-0.572943
15406,low,-0.598699
5998,criminal,-0.689780


In [17]:
frames = [new_popular, new_negative]
result = pd.concat(frames,sort=False)

In [189]:
result.to_csv('top20popular.csv', sep=',')

#### Let's try Advanced Method!

In [18]:
advancedvectorizer = CountVectorizer(ngram_range=(2,2),stop_words='english')
advancedtrain = advancedvectorizer.fit_transform(df['text'])
df_new_2 = pd.DataFrame(advancedtrain.toarray(), columns=advancedvectorizer.get_feature_names())

In [19]:
from sklearn.linear_model import LogisticRegression
advancedmodel = LogisticRegression()
advancedmodel = advancedmodel.fit(advancedtrain, df["Label"])

/Users/zhaoze/anaconda3/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


## Topic Modeling

Topic modeling technique summary:
Topic modeling is a process of discovering 'topics' that occur in a collection of documents(a form of dimensionality reduction). Topic can be thought of as a probability distribution of words. 

- **Matrix Factorization**:
  
  1) *Latent Semantic Analysis (LSA)*: 
  
  $X_{(terms*documents)} = U_{(terms*topics)}\sum_{(topics*topics)}V^{T}_{topics*documents}$
  
  2) *Non-Negative Matrix Factorization (NMF)*:
  
  $V_{(terms*documents)} = W_{(terms*topics)}H_{topics*documents}$
  
    - NMF has the extra constraint of positive values, it will tend to lose more information when truncating.
    - Since NMF can never undo the application of a latent feature, it is much more careful about what it adds at each step. In some applications, this can make for more human interpretable latent features.
    - NMF does not have to give orthogonal latent vectors.
  

- **Probabilistic Modeling**: 
  
  1) *Latent Dirichlet Allocation (LDA)*: 
  Think in terms of probability distributions: every document consists of a distribution of topics, and every topic consists of a distribution of words. 
  
  **How LDA does this?** 
  
  Start with a corpus of documents -> Assume some topic distribution in each document -> Assume some word distribution in each topic -> Look at the corpus and try to find what topic and word distributions would be most likely to generate that data
  
Parameters to tune:
1) Number of topics
2) Text preprocessing
  
Comparison among three techniques:

1) LSA and NMF tend to work better on smaller documents (tweets)

2) LDA tends to work better on larger documents (books)

3) Apply a technique, look at the results, and continuously tweak your parameters until the topics make sense


In [20]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import NMF
from sklearn.metrics.pairwise import cosine_similarity

##### NMF

In [21]:
ex_label = [e[:30]+"..." for e in topic_modeling]
from sklearn.feature_extraction.text import CountVectorizer
            
vectorizer = CountVectorizer(stop_words = 'english')
doc_word = vectorizer.fit_transform(topic_modeling)
pd.DataFrame(doc_word.toarray(), index=ex_label, columns=vectorizer.get_feature_names()).head(1)


,aa,aaa,aaaw,aab,aabo,aadmi,aafia,aali,aam,aamer,aamir,aap,aaron,aarp,ababa,aback,abadan,aban,abandon,abandoned,abandoning,abandonment,abarrel,abating,abattoir,abay,abaya,abba,abbas,abbasi,abbass,abbe,abbey,abbot,abbott,abbottabad,abbotts,abbreviation,abby,abc,abdel,abdelbaset,abdelfattah,abdelrahman,abdeslam,abdeslams,abdicate,abdicates,abdicating,abdication,abdishakur,abdolfattah,abdolmalek,abdomen,abdominal,abduct,abducted,abducting,abduction,abductor,abdul,abdulateef,abdulaziz,abdullah,abdullahi,abe,abebut,abed,abercrombie,aberdeen,aberdeenshire,abergil,aberrant,aberration,abes,abhinav,abhor,abhorrent,abia,abide,abidine,abiding,ability,abim,abir,abject,abkhazia,abkhazian,ablaze,able,abnormal,abnormality,aboard,abolish,abolished,abolishes,abolishing,abolition,abomb,aboriginal,aborigine,abort,aborted,aborting,abortion,abortionrights,abosolve,abou,aboul,abound,abounds,aboutface,aboveaverage,aboveground,abraham,abrahamian,abrahamic,abramoff,abramovay,abrams,abrini,abroad,abrogating,abrupt,abruptly,abscond,abseiling,absence,absent,absentee,absenteeism,absentia,absinthe,absolute,absolutely,absorb,absorbed,absorbing,abstain,abstained,abstaining,abstains,abstention,abstinence,abstinenceonly,abstraction,absurd,absurdity,abu,abubakar,abubakr,abuja,abundance,abundant,abuse,abused,abuseofpower,abuser,abusing,abusive,abydos,abysmal,abyss,ac,academic,academically,academy,acadian,acapulco,acapulcoin,accelerate,accelerated,accelerates,acceleratestemperatures,accelerating,accelerator,accent,accept,acceptable,acceptablelooking,acceptably,acceptance,accepted,accepting,accepts,access,accessed,accessible,accessing,accession,accessory,accident,accidental,accidentally,accidently,acclaim,acclaimed,acclamation,accolade,accommodating,accommodation,accompanied,accompany,accompanying,accomplice,accomplish,accomplished,accomplishment,accomplishmentsthe,accord,accordance,according,accordingly,accosted,account,accountability,accountable,accountancy,accountant,accounted,accountholders,accounting,accreditation,accredited,accrington,accumulated,accumulating,accumulation,accuracy,accurate,accurately,accusation,accuse,accused,accuser,accuses,accusing,accustomed,acdc,acdcs,ace,aceh,acehs,ache,achebe,acheives,achievable,achieve,achieved,achievement,...,yielded,yifu,yikes,yingcong,yingjeou,yingluck,yitzak,yitzhar,yiwu,yle,ynet,ynetnews,yo,yoani,yoda,yodelling,yoga,yogurt,yojong,yold,yom,yomiuri,yong,yongbyon,yongil,yongrim,yonhap,yoo,yoram,york,yorkborn,yorke,yorker,yorkers,yorklondon,yorkshire,yorkshireman,yosef,yoshida,yoshidas,yottayears,youcef,youd,yougov,youguang,youll,younes,young,younger,youngest,youngster,youngsu,younis,younnbanks,youre,yousafza,yousafzai,yousafzais,yousef,youssef,yousuf,yousufzai,youth,youthful,youthsn,youtube,youtubed,youtubes,youve,yoweri,yoyogi,ypf,ypg,yr,yrold,yrolds,yu,yuan,yucatan,yuck,yudhoyono,yue,yugoslav,yugoslavia,yujiao,yulans,yulia,yulin,yum,yunnan,yuppie,yuri,yushchenko,yusuf,yutu,yuval,zabadani,zacarias,zacatecas,zack,zadar,zahi,zahorudin,zahra,zahras,zaibatsu,zaid,zainab,zaire,zakaria,zakayev,zakir,zakiur,zakynthos,zales,zaman,zamani,zambia,zambian,zamboanga,zanardi,zanardis,zandani,zanupf,zanzibar,zapatero,zapatista,zapiro,zapping,zara,zarca,zardari,zarif,zarqawi,zaveri,zawiya,zawiyah,zawraa,zayas,zaynab,zbigniew,zdf,zealand,zealander,zealot,zebra,zedong,zedongs,zeenat,zeid,zeidan,zeit,zeits,zeitung,zeland,zelaya,zelayas,zell,zen,zenroren,zero,zeroed,zeroemissions,zerohour,zerohours,zeroing,zerorupee,zerotolerance,zerowaste,zeta,zeus,zhaleh,zhanaozen,zhang,zhanglidong,zhao,zhaoxu,zheng,zhenghu,zhijun,zhisheng,zhisong,zhongying,zhou,zhuhai,ziad,zidane,zika,zikainfected,zikalinked,zikanegative,zikapositive,zim,zimbabwe,zimbabwean,zimbabwehood,zimbabwestyle,zimbawe,zimmermann,zina,zinc,zindler,zine,zinedine,zinn,zintan,zion,zionazism,zionism,zionist,zip,ziyang,zizek,zlin,zlitan,zliten,zmirs,zoabi,zoe,zogby,zoha,zombie,zone,zong,zoo,zookeeper,zookeepers,zoologist,zoomin,zooming,zor,zossen,zrich,zschpe,zte,zu,zubaydah,zuckb

In [24]:
nmf_model = NMF(10)
doc_topic = nmf_model.fit_transform(doc_word)

In [25]:
topic_word = pd.DataFrame(nmf_model.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5","component_6","component_7","component_8","component_9","component_10"],
             columns = vectorizer.get_feature_names())
topic_word

,aa,aaa,aaaw,aab,aabo,aadmi,aafia,aali,aam,aamer,aamir,aap,aaron,aarp,ababa,aback,abadan,aban,abandon,abandoned,abandoning,abandonment,abarrel,abating,abattoir,abay,abaya,abba,abbas,abbasi,abbass,abbe,abbey,abbot,abbott,abbottabad,abbotts,abbreviation,abby,abc,abdel,abdelbaset,abdelfattah,abdelrahman,abdeslam,abdeslams,abdicate,abdicates,abdicating,abdication,abdishakur,abdolfattah,abdolmalek,abdomen,abdominal,abduct,abducted,abducting,abduction,abductor,abdul,abdulateef,abdulaziz,abdullah,abdullahi,abe,abebut,abed,abercrombie,aberdeen,aberdeenshire,abergil,aberrant,aberration,abes,abhinav,abhor,abhorrent,abia,abide,abidine,abiding,ability,abim,abir,abject,abkhazia,abkhazian,ablaze,able,abnormal,abnormality,aboard,abolish,abolished,abolishes,abolishing,abolition,abomb,aboriginal,aborigine,abort,aborted,aborting,abortion,abortionrights,abosolve,abou,aboul,abound,abounds,aboutface,aboveaverage,aboveground,abraham,abrahamian,abrahamic,abramoff,abramovay,abrams,abrini,abroad,abrogating,abrupt,abruptly,abscond,abseiling,absence,absent,absentee,absenteeism,absentia,absinthe,absolute,absolutely,absorb,absorbed,absorbing,abstain,abstained,abstaining,abstains,abstention,abstinence,abstinenceonly,abstraction,absurd,absurdity,abu,abubakar,abubakr,abuja,abundance,abundant,abuse,abused,abuseofpower,abuser,abusing,abusive,abydos,abysmal,abyss,ac,academic,academically,academy,acadian,acapulco,acapulcoin,accelerate,accelerated,accelerates,acceleratestemperatures,accelerating,accelerator,accent,accept,acceptable,acceptablelooking,acceptably,acceptance,accepted,accepting,accepts,access,accessed,accessible,accessing,accession,accessory,accident,accidental,accidentally,accidently,acclaim,acclaimed,acclamation,accolade,accommodating,accommodation,accompanied,accompany,accompanying,accomplice,accomplish,accomplished,accomplishment,accomplishmentsthe,accord,accordance,according,accordingly,accosted,account,accountability,accountable,accountancy,accountant,accounted,accountholders,accounting,accreditation,accredited,accrington,accumulated,accumulating,accumulation,accuracy,accurate,accurately,accusation,accuse,accused,accuser,accuses,accusing,accustomed,acdc,acdcs,ace,aceh,acehs,ache,achebe,acheives,achievable,achieve,achieved,achievement,...,yielded,yifu,yikes,yingcong,yingjeou,yingluck,yitzak,yitzhar,yiwu,yle,ynet,ynetnews,yo,yoani,yoda,yodelling,yoga,yogurt,yojong,yold,yom,yomiuri,yong,yongbyon,yongil,yongrim,yonhap,yoo,yoram,york,yorkborn,yorke,yorker,yorkers,yorklondon,yorkshire,yorkshireman,yosef,yoshida,yoshidas,yottayears,youcef,youd,yougov,youguang,youll,younes,young,younger,youngest,youngster,youngsu,younis,younnbanks,youre,yousafza,yousafzai,yousafzais,yousef,youssef,yousuf,yousufzai,youth,youthful,youthsn,youtube,youtubed,youtubes,youve,yoweri,yoyogi,ypf,ypg,yr,yrold,yrolds,yu,yuan,yucatan,yuck,yudhoyono,yue,yugoslav,yugoslavia,yujiao,yulans,yulia,yulin,yum,yunnan,yuppie,yuri,yushchenko,yusuf,yutu,yuval,zabadani,zacarias,zacatecas,zack,zadar,zahi,zahorudin,zahra,zahras,zaibatsu,zaid,zainab,zaire,zakaria,zakayev,zakir,zakiur,zakynthos,zales,zaman,zamani,zambia,zambian,zamboanga,zanardi,zanardis,zandani,zanupf,zanzibar,zapatero,zapatista,zapiro,zapping,zara,zarca,zardari,zarif,zarqawi,zaveri,zawiya,zawiyah,zawraa,zayas,zaynab,zbigniew,zdf,zealand,zealander,zealot,zebra,zedong,zedongs,zeenat,zeid,zeidan,zeit,zeits,zeitung,zeland,zelaya,zelayas,zell,zen,zenroren,zero,zeroed,zeroemissions,zerohour,zerohours,zeroing,zerorupee,zerotolerance,zerowaste,zeta,zeus,zhaleh,zhanaozen,zhang,zhanglidong,zhao,zhaoxu,zheng,zhenghu,zhijun,zhisheng,zhisong,zhongying,zhou,zhuhai,ziad,zidane,zika,zikainfected,zikalinked,zikanegative,zikapositive,zim,zimbabwe,zimbabwean,zimbabwehood,zimbabwestyle,zimbawe,zimmermann,zina,zinc,zindler,zine,zinedine,zinn,zintan,zion,zionazism,zionism,zionist,zip,ziyang,zizek,zlin,zlitan,zliten,zmirs,zoabi,zoe,zogby,zoha,zombie,zone,zong,zoo,zookeeper,zookeepers,zoologist,zoomin,zooming,zor,zossen,zrich,zschpe,zte,zu,zubaydah,zuckb

In [26]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [27]:
display_topics(nmf_model, vectorizer.get_feature_names(), 10)


Topic  0
say, new, isi, syria, world, state, year, report, saudi, uk

Topic  1
russia, ukraine, putin, military, ukrainian, crimea, united_states, nato, troop, force

Topic  2
israel, israeli, gaza, palestinian, war, hamas, say, attack, child, soldier

Topic  3
united_states, wikileaks, snowden, government, american, war, wa, pakistan, obama, nsa

Topic  4
korea, north, south, nuclear, missile, kim, launch, news, war, military

Topic  5
china, amp, new, india, world, bank, news, city, woman, country

Topic  6
wa, year, police, people, world, child, iran, war, uk, death

Topic  7
ha, world, country, government, new, minister, million, court, said, law

Topic  8
protest, police, egypt, government, protester, egyptian, people, president, thousand, force

Topic  9
japan, nuclear, plant, wa, fukushima, iran, force, syria, power, reactor


In [28]:
H = pd.DataFrame(doc_topic.round(5),
             index = ex_label,
             columns = ["component_1","component_2","component_3","component_4","component_5","component_6","component_7","component_8","component_9","component_10"])


In [29]:
data.shape

(1989, 33)

In [30]:
basicmodel2 = LogisticRegression(solver='lbfgs')
basicmodel2 = basicmodel2.fit(H, data['Label'])

# X_test = cv.transform(test['text'])
# predictions = basicmodel.predict(X_test)

In [31]:
basicmodel2.coef_

array([[-0.02467751,  0.00816449, -0.00689202, -0.01704895,  0.05697155,
         0.26930782, -0.2310856 , -0.20738771,  0.14845203, -0.21736616]])

In [32]:
df_2 = {'topic': ['Arms Control','Iran','War on Terror','Free Trade and Globalization', 'Human Rights'], 'coeficient': [ -0.22, -0.23, -0.21, 0.27, 0.15]}
df_2 = pd.DataFrame(data=df_2)
df_2

,topic,coeficient
0,Arms Control,-0.22
1,Iran,-0.23
2,War on Terror,-0.21
3,Free Trade and Globalization,0.27
4,Human Rights,0.15


In [205]:
df_2.to_csv('top5topic.csv', sep=',')

### LSA

In [33]:
vectorizer = CountVectorizer(stop_words='english')
doc_word = vectorizer.fit_transform(topic_modeling)
doc_word.shape

(1989, 32844)

In [34]:
ex_label = [e[:30]+"..." for e in topic_modeling]
pd.DataFrame(doc_word.toarray(), index=ex_label, columns=vectorizer.get_feature_names()).head(10)

,aa,aaa,aaaw,aab,aabo,aadmi,aafia,aali,aam,aamer,aamir,aap,aaron,aarp,ababa,aback,abadan,aban,abandon,abandoned,abandoning,abandonment,abarrel,abating,abattoir,abay,abaya,abba,abbas,abbasi,abbass,abbe,abbey,abbot,abbott,abbottabad,abbotts,abbreviation,abby,abc,abdel,abdelbaset,abdelfattah,abdelrahman,abdeslam,abdeslams,abdicate,abdicates,abdicating,abdication,abdishakur,abdolfattah,abdolmalek,abdomen,abdominal,abduct,abducted,abducting,abduction,abductor,abdul,abdulateef,abdulaziz,abdullah,abdullahi,abe,abebut,abed,abercrombie,aberdeen,aberdeenshire,abergil,aberrant,aberration,abes,abhinav,abhor,abhorrent,abia,abide,abidine,abiding,ability,abim,abir,abject,abkhazia,abkhazian,ablaze,able,abnormal,abnormality,aboard,abolish,abolished,abolishes,abolishing,abolition,abomb,aboriginal,aborigine,abort,aborted,aborting,abortion,abortionrights,abosolve,abou,aboul,abound,abounds,aboutface,aboveaverage,aboveground,abraham,abrahamian,abrahamic,abramoff,abramovay,abrams,abrini,abroad,abrogating,abrupt,abruptly,abscond,abseiling,absence,absent,absentee,absenteeism,absentia,absinthe,absolute,absolutely,absorb,absorbed,absorbing,abstain,abstained,abstaining,abstains,abstention,abstinence,abstinenceonly,abstraction,absurd,absurdity,abu,abubakar,abubakr,abuja,abundance,abundant,abuse,abused,abuseofpower,abuser,abusing,abusive,abydos,abysmal,abyss,ac,academic,academically,academy,acadian,acapulco,acapulcoin,accelerate,accelerated,accelerates,acceleratestemperatures,accelerating,accelerator,accent,accept,acceptable,acceptablelooking,acceptably,acceptance,accepted,accepting,accepts,access,accessed,accessible,accessing,accession,accessory,accident,accidental,accidentally,accidently,acclaim,acclaimed,acclamation,accolade,accommodating,accommodation,accompanied,accompany,accompanying,accomplice,accomplish,accomplished,accomplishment,accomplishmentsthe,accord,accordance,according,accordingly,accosted,account,accountability,accountable,accountancy,accountant,accounted,accountholders,accounting,accreditation,accredited,accrington,accumulated,accumulating,accumulation,accuracy,accurate,accurately,accusation,accuse,accused,accuser,accuses,accusing,accustomed,acdc,acdcs,ace,aceh,acehs,ache,achebe,acheives,achievable,achieve,achieved,achievement,...,yielded,yifu,yikes,yingcong,yingjeou,yingluck,yitzak,yitzhar,yiwu,yle,ynet,ynetnews,yo,yoani,yoda,yodelling,yoga,yogurt,yojong,yold,yom,yomiuri,yong,yongbyon,yongil,yongrim,yonhap,yoo,yoram,york,yorkborn,yorke,yorker,yorkers,yorklondon,yorkshire,yorkshireman,yosef,yoshida,yoshidas,yottayears,youcef,youd,yougov,youguang,youll,younes,young,younger,youngest,youngster,youngsu,younis,younnbanks,youre,yousafza,yousafzai,yousafzais,yousef,youssef,yousuf,yousufzai,youth,youthful,youthsn,youtube,youtubed,youtubes,youve,yoweri,yoyogi,ypf,ypg,yr,yrold,yrolds,yu,yuan,yucatan,yuck,yudhoyono,yue,yugoslav,yugoslavia,yujiao,yulans,yulia,yulin,yum,yunnan,yuppie,yuri,yushchenko,yusuf,yutu,yuval,zabadani,zacarias,zacatecas,zack,zadar,zahi,zahorudin,zahra,zahras,zaibatsu,zaid,zainab,zaire,zakaria,zakayev,zakir,zakiur,zakynthos,zales,zaman,zamani,zambia,zambian,zamboanga,zanardi,zanardis,zandani,zanupf,zanzibar,zapatero,zapatista,zapiro,zapping,zara,zarca,zardari,zarif,zarqawi,zaveri,zawiya,zawiyah,zawraa,zayas,zaynab,zbigniew,zdf,zealand,zealander,zealot,zebra,zedong,zedongs,zeenat,zeid,zeidan,zeit,zeits,zeitung,zeland,zelaya,zelayas,zell,zen,zenroren,zero,zeroed,zeroemissions,zerohour,zerohours,zeroing,zerorupee,zerotolerance,zerowaste,zeta,zeus,zhaleh,zhanaozen,zhang,zhanglidong,zhao,zhaoxu,zheng,zhenghu,zhijun,zhisheng,zhisong,zhongying,zhou,zhuhai,ziad,zidane,zika,zikainfected,zikalinked,zikanegative,zikapositive,zim,zimbabwe,zimbabwean,zimbabwehood,zimbabwestyle,zimbawe,zimmermann,zina,zinc,zindler,zine,zinedine,zinn,zintan,zion,zionazism,zionism,zionist,zip,ziyang,zizek,zlin,zlitan,zliten,zmirs,zoabi,zoe,zogby,zoha,zombie,zone,zong,zoo,zookeeper,zookeepers,zoologist,zoomin,zooming,zor,zossen,zrich,zschpe,zte,zu,zubaydah,zuckb

In [35]:
lsa = TruncatedSVD(10)
doc_topic = lsa.fit_transform(doc_word)
lsa.explained_variance_ratio_

array([0.00705729, 0.01647397, 0.01376462, 0.01153813, 0.01056789,
       0.00774614, 0.00754892, 0.00687036, 0.00630044, 0.00572034])

In [36]:
topic_word = pd.DataFrame(lsa.components_.round(3),
             index = ["component_1","component_2","component_3","component_4","component_5","component_6","component_7","component_8","component_9","component_10"],
             columns = vectorizer.get_feature_names())
topic_word

,aa,aaa,aaaw,aab,aabo,aadmi,aafia,aali,aam,aamer,aamir,aap,aaron,aarp,ababa,aback,abadan,aban,abandon,abandoned,abandoning,abandonment,abarrel,abating,abattoir,abay,abaya,abba,abbas,abbasi,abbass,abbe,abbey,abbot,abbott,abbottabad,abbotts,abbreviation,abby,abc,abdel,abdelbaset,abdelfattah,abdelrahman,abdeslam,abdeslams,abdicate,abdicates,abdicating,abdication,abdishakur,abdolfattah,abdolmalek,abdomen,abdominal,abduct,abducted,abducting,abduction,abductor,abdul,abdulateef,abdulaziz,abdullah,abdullahi,abe,abebut,abed,abercrombie,aberdeen,aberdeenshire,abergil,aberrant,aberration,abes,abhinav,abhor,abhorrent,abia,abide,abidine,abiding,ability,abim,abir,abject,abkhazia,abkhazian,ablaze,able,abnormal,abnormality,aboard,abolish,abolished,abolishes,abolishing,abolition,abomb,aboriginal,aborigine,abort,aborted,aborting,abortion,abortionrights,abosolve,abou,aboul,abound,abounds,aboutface,aboveaverage,aboveground,abraham,abrahamian,abrahamic,abramoff,abramovay,abrams,abrini,abroad,abrogating,abrupt,abruptly,abscond,abseiling,absence,absent,absentee,absenteeism,absentia,absinthe,absolute,absolutely,absorb,absorbed,absorbing,abstain,abstained,abstaining,abstains,abstention,abstinence,abstinenceonly,abstraction,absurd,absurdity,abu,abubakar,abubakr,abuja,abundance,abundant,abuse,abused,abuseofpower,abuser,abusing,abusive,abydos,abysmal,abyss,ac,academic,academically,academy,acadian,acapulco,acapulcoin,accelerate,accelerated,accelerates,acceleratestemperatures,accelerating,accelerator,accent,accept,acceptable,acceptablelooking,acceptably,acceptance,accepted,accepting,accepts,access,accessed,accessible,accessing,accession,accessory,accident,accidental,accidentally,accidently,acclaim,acclaimed,acclamation,accolade,accommodating,accommodation,accompanied,accompany,accompanying,accomplice,accomplish,accomplished,accomplishment,accomplishmentsthe,accord,accordance,according,accordingly,accosted,account,accountability,accountable,accountancy,accountant,accounted,accountholders,accounting,accreditation,accredited,accrington,accumulated,accumulating,accumulation,accuracy,accurate,accurately,accusation,accuse,accused,accuser,accuses,accusing,accustomed,acdc,acdcs,ace,aceh,acehs,ache,achebe,acheives,achievable,achieve,achieved,achievement,...,yielded,yifu,yikes,yingcong,yingjeou,yingluck,yitzak,yitzhar,yiwu,yle,ynet,ynetnews,yo,yoani,yoda,yodelling,yoga,yogurt,yojong,yold,yom,yomiuri,yong,yongbyon,yongil,yongrim,yonhap,yoo,yoram,york,yorkborn,yorke,yorker,yorkers,yorklondon,yorkshire,yorkshireman,yosef,yoshida,yoshidas,yottayears,youcef,youd,yougov,youguang,youll,younes,young,younger,youngest,youngster,youngsu,younis,younnbanks,youre,yousafza,yousafzai,yousafzais,yousef,youssef,yousuf,yousufzai,youth,youthful,youthsn,youtube,youtubed,youtubes,youve,yoweri,yoyogi,ypf,ypg,yr,yrold,yrolds,yu,yuan,yucatan,yuck,yudhoyono,yue,yugoslav,yugoslavia,yujiao,yulans,yulia,yulin,yum,yunnan,yuppie,yuri,yushchenko,yusuf,yutu,yuval,zabadani,zacarias,zacatecas,zack,zadar,zahi,zahorudin,zahra,zahras,zaibatsu,zaid,zainab,zaire,zakaria,zakayev,zakir,zakiur,zakynthos,zales,zaman,zamani,zambia,zambian,zamboanga,zanardi,zanardis,zandani,zanupf,zanzibar,zapatero,zapatista,zapiro,zapping,zara,zarca,zardari,zarif,zarqawi,zaveri,zawiya,zawiyah,zawraa,zayas,zaynab,zbigniew,zdf,zealand,zealander,zealot,zebra,zedong,zedongs,zeenat,zeid,zeidan,zeit,zeits,zeitung,zeland,zelaya,zelayas,zell,zen,zenroren,zero,zeroed,zeroemissions,zerohour,zerohours,zeroing,zerorupee,zerotolerance,zerowaste,zeta,zeus,zhaleh,zhanaozen,zhang,zhanglidong,zhao,zhaoxu,zheng,zhenghu,zhijun,zhisheng,zhisong,zhongying,zhou,zhuhai,ziad,zidane,zika,zikainfected,zikalinked,zikanegative,zikapositive,zim,zimbabwe,zimbabwean,zimbabwehood,zimbabwestyle,zimbawe,zimmermann,zina,zinc,zindler,zine,zinedine,zinn,zintan,zion,zionazism,zionism,zionist,zip,ziyang,zizek,zlin,zlitan,zliten,zmirs,zoabi,zoe,zogby,zoha,zombie,zone,zong,zoo,zookeeper,zookeepers,zoologist,zoomin,zooming,zor,zossen,zrich,zschpe,zte,zu,zubaydah,zuckb

In [37]:
def display_topics(model, feature_names, no_top_words, topic_names=None):
    for ix, topic in enumerate(model.components_):
        if not topic_names or not topic_names[ix]:
            print("\nTopic ", ix)
        else:
            print("\nTopic: '",topic_names[ix],"'")
        print(", ".join([feature_names[i]
                        for i in topic.argsort()[:-no_top_words - 1:-1]]))

In [38]:
display_topics(lsa, vectorizer.get_feature_names(), 10)


Topic  0
united_states, ha, say, china, russia, wa, year, world, new, government

Topic  1
russia, ukraine, say, united_states, putin, isi, syria, crimea, ukrainian, snowden

Topic  2
israel, gaza, israeli, united_states, russia, palestinian, ukraine, war, hamas, soldier

Topic  3
united_states, korea, north, wikileaks, south, assange, snowden, american, cable, pakistan

Topic  4
korea, north, russia, south, israel, gaza, israeli, ukraine, china, nuclear

Topic  5
china, russia, police, wa, protest, japan, iran, amp, protester, people

Topic  6
ha, russia, protest, wikileaks, wa, egypt, police, iran, ukraine, protester

Topic  7
say, police, egypt, protest, protester, korea, egyptian, people, syria, north

Topic  8
say, protest, ha, china, egypt, government, japan, gaza, protester, egyptian

Topic  9
japan, wa, syria, nuclear, attack, iran, isi, killed, ha, plant
